In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from re import sub, search, findall
import os
import pandas as pd
import numpy as np
from glob import glob
def remover(file_):
    file_ = ''.join(file_.split()).lower()
    stop_element = ['менее|<|>|автоматическая|ручная|[()]'
              '|меньш.|больше|более|до(?=[0-9])|менее|iultra|от(?=[0-9])'] #'до' ищет только после чисел, тк идет захват 'биохимисслеДОваниекрови'. аспартатаминОТрансфераза = аспартатаминрансфераза
    for elem in stop_element:
      file_ = sub(elem, '', file_)
    file_ = sub(r',', '.', file_) # иначе не меняет на точку 
    return(file_)
def change_date_format(dt):
    """ на первом этапе переводим "нормальный формат" в формат через дефисы
     на втором этапе ищем вариант дд.мм.гг в формат через дефисы"""
    data_1 =  sub(r'(0?[1-9]|[12]\d|30|31)[.](0?[1-9]|1[0-2])[.](\d{4})', 
                     '\\1-\\2-\\3', dt)
    return sub(r'(0?[1-9]|[12]\d|30|31)[.](0?[1-9]|1[0-2])[.](\d{2})', 
                  '\\1-\\2-20\\3', data_1)

In [3]:
lst = list(glob(os.path.join('/content/drive/MyDrive/parser', '*')))

In [10]:
test_dates = []
patients_id = []
patients_birthday = []
values_wbc, values_plt = [], []
values_mono, values_lymph, values_neut = [], [], []
for doc in lst:
  with open(doc, encoding='utf-8', errors='ignore') as file_:
#with open('/content/drive/MyDrive/проверка/германович-6.txt') as file_:
    file_ = change_date_format(remover(file_.read()))
    file_1 = sub(r'(общийанализкрови)(\d{2}-\d{2}-\d{4})', r'\2\1', file_)
    for value in findall(r'(\d{2}-\d{2}-\d{4}общийанализкрови.*?)(?=\d{2}-\d{2}-\d{4})', file_1):
      date = search(r'(\d{2}-\d{2}-\d{4})(?=общийанализкрови)', file_1)
      if date:
        test_dates.append(date[0])
      id = search(r'\d+', file_1) #ищем айди. находит самую первую дату в эпикризе. !!! есть эпикризы в которых первые цифры это номер отделения, типа онко отд. №2
      if id:
        patients_id.append(id[0])
      birthday = search("(\d{2})-(\d{2})-(\d{4})", file_1)
      if birthday:
        patients_birthday.append(birthday[0])
      wbc = search(r'(?<=лейкоциты)\d+(|.\d+)(?=10.9)', value)
      if wbc:
        values_wbc.append(wbc[0])
      else:
        values_wbc.append('NaN')
      plt = search(r'(?<=тромбоциты)\d+(?=10.9)', value)
      if plt:
        values_plt.append(plt[0])
      else:
        values_plt.append('NaN')
      mono = search(r'(?<=моноциты)\d+', value)
      if mono:
        values_mono.append(mono[0])
      else:
        values_mono.append('NaN')
      lymph = search(r'(?<=лимфоциты)\d+', value)
      if lymph:
        values_lymph.append(lymph[0])
      else:
        values_lymph.append('NaN')
      neut = search(r'(?<=нейтрофилы)\d+', value)
      if neut:
        values_neut.append(neut[0])
      else:
        values_neut.append('NaN')

In [11]:
data_dct = {'ID':patients_id, 'Birthday':patients_birthday, 'dates':test_dates, 'wbc':values_wbc, 'plt':values_plt, 'neut':values_neut, 'lymph':values_lymph, 'mono':values_mono}
df = pd.DataFrame(data_dct)
df = df.replace(r'', np.nan, regex=True)
df

,ID,Birthday,dates,wbc,plt,neut,lymph,mono
0,1025,19-01-2000,13-02-2020,10.3,188,NaN,NaN,NaN
1,1025,19-01-2000,13-02-2020,8.41,237,53,32,12
2,1025,19-01-2000,13-02-2020,7.54,190,NaN,28,15
3,6112,25-09-1940,16-10-2019,8.23,155,NaN,10,5
4,7658,24-08-1993,11-12-2018,8.55,130,NaN,29,6
...,...,...,...,...,...,...,...,...
2974,2676,29-06-1981,11-06-2020,8.34,190,NaN,24,6
2975,2676,29-06-1981,11-06-2020,4.42,248,33,40,14
2976,6202,06-03-1992,22-10-2019,10.39,216,69,23,6
2977,6202,06-03-1992,22-10-2019,11.45,257,69,23,610
